# Recon Configuration Automation
### 1.recon table

In [ ]:
import pandas as pd
import numpy as np
import os
df = pd.read_excel('Baroda_CID Rules.xlsx')
df.fillna(value='',inplace=True)
df = df[df['Type'] == 'Feed']
rules_df = pd.read_excel('Baroda_CID Rules.xlsx',sheet_name = 'SourceVsTask')
rules_df.fillna(value='',inplace=True)
df_obj = rules_df.select_dtypes(['object'])
rules_df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
rules_df['Fullmatch_Condition'] = rules_df['Fullmatch_Condition'].replace(['=='],'=')
rules_df['Crosscredit_Condition'] = rules_df['Crosscredit_Condition'].replace(['=='],'=')

In [ ]:
recon = pd.DataFrame(columns=["ID", "Recon_ID", "Raw_Feed", "Left_Table", "Raw_Join", "Right_Table", "Join_Condition", "Column_Selection", "Recon_Update", "Batch", "module_name"])

In [ ]:
def create_joincond(Sub_Feed, feed_df, type_):
    join_str = 'SELECT Z.*, O.* FROM standard_feed AS Z INNER JOIN standard_task AS O ON '
    if type_ == 'feedvstask':
        for idx, rule1 in feed_df.iterrows():
            tem_str = f"Z.`{rule1['Fullmatch_Source_Column']}`{rule1['Fullmatch_Condition']}O.`{rule1['Fullmatch_Task_Column']}` {rule1['Condition']} "
            #Z.`ORDER_NO`=O.`batch` AND Z.`RTA_SCH_CODE`=O.`scheme` AND Z.`AMOUNT`=O.`amt`
            join_str = join_str+tem_str
        join_str = join_str.strip().rstrip('AND')
        return join_str
    else:
        for idx, rule1 in feed_df.iterrows():
            tem_str = f"Z.`{rule1['Fullmatch_Source_Column']}`{rule1['Fullmatch_Condition']}O.`{rule1['Fullmatch_Task_Column']}` {rule1['Condition']} "
            #Z.`ORDER_NO`=O.`batch` AND Z.`RTA_SCH_CODE`=O.`scheme` AND Z.`AMOUNT`=O.`amt`
            join_str = join_str+tem_str
        join_str = join_str.strip().rstrip('AND').strip()
        return join_str

In [ ]:
row_val = 0
for index, row in df.iterrows():
    print(row['Sub-Feed'])
    Sub_Feed = row['Sub-Feed']
    table_name = row['Table-Name']
    module_name = row['module_name']
    #  Creating Ingestion Folders
    #os.mkdir(f"SharedFolders/{row['Sub-Feed']}")
    feed_df = rules_df[rules_df['Name (Sub_feed/Bank)'] == Sub_Feed]
    Raw_Join = f'SELECT Z.*, O.`Feed_ID` FROM {table_name} AS Z RIGHT JOIN standard_feed AS O ON Z.`Raw_ID`=O.`Feed_ID`'
    join_condition = create_joincond(Sub_Feed, feed_df, type_='feedvstask')
    update_quer = f"ALTER TABLE [Tem] ALTER COLUMN [Raw_ID] [varchar](100) NULL;UPDATE [standard_task] SET [standard_task].[Matched_Feed_ID] = [Tem].[Raw_ID] , [standard_task].[reconciled_datetime] = GETDATE(),[standard_task].[Credit Received Date] = [standard_feed].[Credit Received Date] ,[standard_task].[ID]=[standard_feed].[ID],[standard_task].[Credit Received Time] = [standard_feed].[Credit Received Time], [standard_task].[Bank_Name] = [standard_feed].[Bank_Name], [standard_task].[Bank_Account_Number] = [standard_feed].[Bank_Account_Number], [standard_task].[queue] = 'Tran_Query' FROM [standard_task], [standard_feed], [Tem] WHERE [standard_feed].[Feed_ID] = [Tem].[Raw_ID] AND [standard_task].[Task_ID] = [Tem].[Task_ID] AND [standard_task].[Matched_Feed_ID] = '' AND [standard_feed].[Sub_Feed] = '{Sub_Feed}';DELETE FROM [Tem] WHERE [Raw_ID] IN (SELECT X.[Raw_ID] from (SELECT [Raw_ID],[Task_ID] FROM [Tem] EXCEPT SELECT [Matched_Feed_ID],[Task_ID] FROM [standard_task]) as X);UPDATE [standard_feed] SET [standard_feed].[Matched] = 1,[standard_feed].[queue] = 'Tran_Query' FROM [standard_feed], [Tem] WHERE [standard_feed].[Feed_ID] = [Tem].[Raw_ID] AND [standard_feed].[Sub_Feed] = '{Sub_Feed}';UPDATE [{table_name}] SET [{table_name}].[Matched] = 1,[{table_name}].[queue] = 'Tran_Query'  FROM [{table_name}], [Tem] WHERE [{table_name}].[Raw_ID] = [Tem].[Raw_ID];DROP TABLE [Tem]"
    tem_df = {'ID':row_val+1,
              'Recon_ID':'FeedvsTask1',
              'Raw_Feed':table_name,
              'Left_Table':'standard_feed',
              'Raw_Join':Raw_Join,
              'Right_Table':'standard_task',
              'Join_Condition':join_condition,
              'Column_Selection':'Raw_ID,Task_ID',
              'Recon_Update':update_quer,
              'Batch':Sub_Feed.lower(),
              'module_name':module_name}
    recon = recon.append(tem_df, ignore_index = True)
    
    join_condition2 = create_joincond(Sub_Feed, feed_df, type_='crosscredit')
    update_quer2 = f"ALTER TABLE [Tem] ALTER COLUMN [Raw_ID] [varchar](100) NULL;UPDATE [standard_task] SET [standard_task].[Feed_ID] = [Tem].[Raw_ID], [standard_task].[reconciled_datetime] = GETDATE() FROM [standard_task], [Tem] WHERE [standard_task].[Task_ID] = [Tem].[Task_ID];UPDATE [standard_task] SET [standard_task].[queue] = 'Exception Reconciliation_Cross Credit' FROM [standard_task], [Tem] WHERE [standard_task].[Task_ID] = [Tem].[Task_ID];UPDATE [standard_feed] SET [standard_feed].[queue] = 'Exception Reconciliation_Cross Credit' FROM [standard_feed], [Tem] WHERE [standard_feed].[Sub_Feed] = '{Sub_Feed}' AND [standard_feed].[Feed_ID] = [Tem].[Raw_ID];UPDATE [{table_name}] SET [{table_name}].[queue] = 'Exception Reconciliation_Cross Credit' FROM [{table_name}], [Tem] WHERE [{table_name}].[Raw_ID] = [Tem].[Raw_ID];DROP TABLE [Tem]"
    tem_df = {'ID':row_val+2,
              'Recon_ID':'Cross_Credit1',
              'Raw_Feed':table_name,
              'Left_Table':'standard_feed',
              'Raw_Join':Raw_Join,
              'Right_Table':'standard_task',
              'Join_Condition':join_condition2,
              'Column_Selection':'Raw_ID,Task_ID',
              'Recon_Update':update_quer2,
              'Batch':Sub_Feed.lower(),
              'module_name':module_name}
    recon = recon.append(tem_df, ignore_index = True)
    row_val+=2

In [ ]:
recon.set_index('ID').to_excel('recon.xlsx',sheet_name='recon')